In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x64367e0
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.ildStyle.SetOptStat(0)

In [4]:
%%cpp
using namespace ROOT::VecOps;

In [5]:
df = ROOT.RDataFrame("events", "data/gen/sw_sl/E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.edm4hep.root")

In [6]:
df.Describe()

Dataframe from TChain events in file data/gen/sw_sl/E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.edm4hep.root

Property                Value
--------                -----
Columns in total           67
Columns from defines        0
Event loops run             0
Processing slots            1

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                   ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeader.weight                      ROOT::VecOps::RVec<Double_t>                    Dataset
EventHeader.weights_begin       

In [7]:
tmp = [f"MCParticle.{s}" for s in ("momentum.x", "momentum.y", "momentum.z", "mass")]
tmp2 = ", ".join(tmp)
print(tmp2)

MCParticle.momentum.x, MCParticle.momentum.y, MCParticle.momentum.z, MCParticle.mass


In [8]:
df = df.Define("good_electrons", "abs(MCParticle.PDG) == 11 && MCParticle.generatorStatus == 1")

In [9]:
# urgh of course I can also have electrons from the jets meh, make this better in the future
df = df.Filter("Sum(good_electrons) == 1", "one electron only")
# df = df.Define("num_good_e", "Sum(good_electrons)")

In [10]:
df = df.Define("P", "sqrt(pow(MCParticle.momentum.x[good_electrons], 2) + pow(MCParticle.momentum.y[good_electrons], 2) + pow(MCParticle.momentum.z[good_electrons], 2))")

In [11]:
df = df.Define("Theta", "atan2(hypot(MCParticle.momentum.x[good_electrons], MCParticle.momentum.y[good_electrons]), MCParticle.momentum.z[good_electrons])")

In [12]:
df = df.Define("cosTheta", "abs(cos(Theta))")

In [13]:
lumi_cone = 0.150

In [14]:
h_P = df.Histo1D("P")
h_Theta = df.Histo1D("Theta")
h_cosTheta_P = df.Histo2D(("", "; #theta; abs(P) [GeV]", 50, 0, ROOT.Math.Pi(), 31, 0, 124), "Theta", "P")

In [15]:
c_cosTheta_P = ROOT.TCanvas()
h_cosTheta_P.Draw("box")

ll = ROOT.TLine(lumi_cone, 0, lumi_cone, 124)
ll.SetLineColor(ROOT.kRed)
lr = ROOT.TLine(ROOT.Math.Pi() - lumi_cone, 0, ROOT.Math.Pi() - lumi_cone, 124)
lr.SetLineColor(ROOT.kRed)
ll.Draw()
lr.Draw()

c_cosTheta_P.Draw()

In [16]:
df2 = ROOT.RDataFrame("events", "data/gen/szeorsw_l/E250-SetA.P4f_szeorsw_l.Gwhizard-2_8_5.eL.pR.I500126.0.edm4hep.root")

In [17]:

df2 = df2.Define("good_electrons", "abs(MCParticle.PDG) == 11 && MCParticle.generatorStatus == 1")
df2 = df2.Filter("Sum(good_electrons) == 2", "require both electrons")
df2 = df2.Define("P", "sqrt(pow(MCParticle.momentum.x[good_electrons], 2) + pow(MCParticle.momentum.y[good_electrons], 2) + pow(MCParticle.momentum.z[good_electrons], 2))")
df2 = df2.Define("Theta", "atan2(hypot(MCParticle.momentum.x[good_electrons], MCParticle.momentum.y[good_electrons]), MCParticle.momentum.z[good_electrons])")

In [21]:
h2_cosTheta_P = df2.Histo2D(("", "; #theta [rad]; abs(P) [GeV]", 50, 0, ROOT.Math.Pi(), 31, 0, 124), "Theta", "P")

In [24]:

c2_cosTheta_P = ROOT.TCanvas()
h2 = h2_cosTheta_P.Clone()
h2.Scale(1 / h2.Integral())
h2.SetLineColor(ROOT.kBlue)
h2.Draw("box")

h = h_cosTheta_P.Clone()
h.Scale(1 / h.Integral())
h.SetLineColor(ROOT.kRed)
h.Draw("box same")

ll = ROOT.TLine(lumi_cone, 0, lumi_cone, 124)
ll.SetLineColor(ROOT.kBlack)
lr = ROOT.TLine(ROOT.Math.Pi() - lumi_cone, 0, ROOT.Math.Pi() - lumi_cone, 124)
lr.SetLineColor(ROOT.kBlack)
ll.Draw()
lr.Draw()

c2_cosTheta_P.SetTitle("sw_sl (red) vs szeorsw_l (blue)")
c2_cosTheta_P.Draw()
c2_cosTheta_P.SaveAs("plots/sw_sl_v_szeorsw_l.pdf")

Info in <TCanvas::Print>: pdf file plots/sw_sl_v_szeorsw_l.pdf has been created
